In [ ]:
def clean_locations(locations, country_list):
    # Create a copy of the locations
    new_locations = locations.copy()
    for i, loc in enumerate(locations):
        found_country = False # Initialize flag for found country to false
        for country in country_list:
            if country in loc: # If a country name is found in the location string
                if (loc.startswith(country + ",") or loc.endswith("," + country) or loc == country): # Check for exact match or match with comma prefix/postfix
                    locations[i] = country # Replace location with the country name
                    found_country = True # Set flag to true
                    break # Exit the inner loop
        """if not found_country:
            print(f"No country found in record {i}: {loc}") # Print message if no country is found"""
    df['Location'] = df['Location'].replace('Türkiye', 'Turkey')
    return new_locations


In [ ]:
# Export the new data frame to a CSV file
df.to_csv("C:/Users/ShaiS/Documents/data_science_project_weather_forcast/sorted_data_after_FACO.csv", index=False)

In [ ]:
# Find the duplicated rows
duplicates = df[df.duplicated()]

# Print the duplicated rows
print(duplicates)

In [ ]:
# Drop duplicates from the combined data frame
df = df.drop_duplicates()

# Find the duplicated rows
duplicates = df[df.duplicated()]

# Print the duplicated rows
print(duplicates)

#RESOLVED - Empty DataFrame - No Dup being found!

In [ ]:
 # Line chart of temperature by hemisphere
    sns.lineplot(x='date', y='Temperature', hue='Serialized Hemisphere', data=df)
    plt.title('Average Temperature by Hemisphere over Time')
    plt.xlabel('Date')
    plt.ylabel('Temperature')
    plt.savefig('Line_chart_of_temperature_by_hemisphere.png', dpi=300)
    plt.clf()

    # Heat map of temperature by city and hemisphere
    heatmap_df = df.pivot('Location', 'Serialized Hemisphere', 'Temperature')
    sns.heatmap(heatmap_df, cmap='coolwarm')
    plt.title('Temperature Variations across Cities in Different Hemispheres')
    plt.xlabel('Serialized Hemisphere')
    plt.ylabel('Location')
    plt.savefig('Heat_map_of_temperature_by_city_and_hemisphere.png', dpi=300)
    plt.clf()

    # Box plot of temperature by hemisphere
    sns.boxplot(x='Serialized Hemisphere', y='Temperature', data=df)
    plt.title('Temperature Distributions by Hemisphere')
    plt.xlabel('Serialized Hemisphere')
    plt.ylabel('Temperature')
    plt.savefig('Box_plot_of_temperature_by_hemisphere.png', dpi=300)
    plt.clf()

    # Scatter plot of temperature by time of year
    df['Month'] = df['date'].dt.month
    sns.scatterplot(x='Month', y='Temperature', data=df)
    plt.title('Temperature vs. Time of Year')
    plt.xlabel('Month')
    plt.ylabel('Temperature')
    plt.savefig('Scatter_plot_of_temperature_by_time_of_year.png', dpi=300)
    plt.clf()

In [ ]:
#ONLY IF YOU WANT TO ASSGIN INDEX FOR TESTING MESSURES
eng_df = eng_df.reset_index().assign(index_col=df.index)
eng_df

In [ ]:
def get_weather_data_for_dates7(start_year, start_month, start_day, end_year, end_month, end_day, browser):

    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            if not browser.current_window_handle: # Check if browser is still open
                browser.quit()
                time.sleep(15)
                browser = webdriver.Firefox()
                browser.maximize_window()
            date_data = get_weather_data4(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 1 # usually on - 5 tries 
            retries = 0
            while retries < max_retries:
                try:
                    browser.quit()
                    time.sleep(15)
                    browser = webdriver.Firefox()
                    browser.maximize_window()
                    date_data = get_weather_data4(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
                    logging.error("An error occurred while getting data for date %s: %s", current_date, e)
                except SessionNotFoundException:
                    browser.quit()
                    time.sleep(15)
                    browser = webdriver.Firefox()
                    browser.maximize_window()
                    date_data = get_weather_data4(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
            if retries == max_retries:
                current_url = browser.current_url
                browser.quit()
                logging.error("Maximum number of retries reached for date %s", current_date)
                problematic_urls.append((current_date.strftime("%Y-%m-%d"), current_url))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
# MULTI THREADED APPROCH FAILS:
import concurrent.futures
from datetime import datetime, timedelta
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

def get_weather_data_for_dates(start_year, start_month, start_day, end_year, end_month, end_day):
    weather_data = {}
    problematic_urls = []
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        while current_date <= end_date:
            year, month, day = current_date.year, current_date.month, current_date.day
            executor.submit(get_weather_data_for_date, year, month, day, weather_data, problematic_urls)
            current_date += timedelta(days=1)
    executor.shutdown(wait=True)
    return weather_data, problematic_urls

def get_weather_data_for_date(year, month, day, weather_data, problematic_urls):
    try:
        browser = webdriver.Firefox()
        date_data = get_weather_data(year, month, day, browser)
        weather_data[current_date.strftime("%Y-%m-%d")] = date_data
    except Exception as e:
        max_retries = 5
        retries = 0
        while retries < max_retries:
            try:
                browser.quit()
                browser = webdriver.Firefox()
                date_data = get_weather_data(year, month, day, browser)
                weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                break
            except Exception as e:
                retries += 1
                time.sleep(30)
                logging.error("An error occurred while getting data for date %s: %s", current_date, e)
        if retries == max_retries:
            current_url = browser.current_url
            browser.quit()
            logging.error("Maximum number

In [ ]:
def get_weather_data3( year, month, day, browser):
    
    # Open browser and navigate to the website
    #browser = webdriver.Chrome()
    browser.get('https://www.wunderground.com/history/daily/il/haifa/LLHA')
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, 'yearSelection')))
    
    # Select year, month, and day from the drop-down menus
    year_select = Select(browser.find_element(By.ID,'yearSelection'))
    year_select.select_by_value(str((2023-year))+':'+' '+str(year))
    month_select = Select(browser.find_element(By.ID,'monthSelection'))
    month_select.select_by_value(str(month))
    day_select = Select(browser.find_element(By.ID,'daySelection'))
    day_select.select_by_value(str((day-1))+':'+' '+str(day))

    # Click the "View" button to load the data for the selected date
    browser.find_element(By.ID, 'dateSubmit').click()
   
    # Wait for the data to load
    WebDriverWait(browser, 20).until_not(EC.presence_of_element_located((By.XPATH, '//div[@class="loading-indicator"]')))
    WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'row')))
    
   
   
    WebDriverWait(browser, 200).until(EC.presence_of_element_located((By.XPATH, '//table[@class="mat-table cdk-table mat-sort ng-star-inserted"]')))
    WebDriverWait(browser, 200)
    
    # Extract the data from the table using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
 
    table = soup.select_one('table[class*="mat-table cdk-table mat-sort ng-star-inserted"]')
    rows = table.find_all('tr',{'class':'mat-row cdk-row ng-star-inserted'})
  
  
    time = []
    temperature = []
    precipitation = []
    wind = []
    
    for row in rows[1:len(rows)-1]:
        cells = row.find_all('td')
        time.append(cells[0].text)
        temperature_element = cells[1].find('span', class_='wu-value wu-value-to')
        temperature.append(temperature_element.text)
        precipitation_element = cells[8].find('span', class_='wu-value wu-value-to')
        precipitation.append(precipitation_element.text)
        wind_element = cells[5].find('span', class_='wu-value wu-value-to')
        wind.append(wind_element.text)
        
   
    #calculating avg
    temperature_avg = fahrenheit_to_celsius(sum(float(t) for t in temperature) / len(temperature))
    precipitation_avg = sum(float(p) for p in precipitation) / len(precipitation)
    wind_avg = sum(float(w) for w in wind) / len(wind)
   

    # Close the browser
    #browser.quit()

   
    data = {'Year': year, 'Month': month, 'Day': day, 'Temperature': temperature_avg, 'Precipitation': precipitation_avg, 'Wind': wind_avg}
    return data
    
   	def fahrenheit_to_celsius(temp_fahrenheit):
    temp_celsius = (temp_fahrenheit - 32) * 5/9
    return temp_celsius

In [ ]:
def get_weather_data_for_dates8(start_year, start_month, start_day, end_year, end_month, end_day, browser):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except SessionNotFoundException:
            browser.quit()
            browser = webdriver.Firefox()
            date_data = get_weather_data3(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    if not browser.current_window_handle: # Check if browser is still open
                        browser.quit()
                        browser = webdriver.Firefox()
                    date_data = get_weather_data3(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
                    logging.error("An error occurred while getting data for date %s: %s", current_date, e)
            if retries == max_retries:
                browser.quit()
                logging.error("Maximum number of retries reached for date %s", current_date)
                problematic_urls.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
def get_weather_data_for_dates6(start_year, start_month, start_day, end_year, end_month, end_day, browser):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            if not browser.current_window_handle: # Check if browser is still open
                browser.quit()
                browser = webdriver.Firefox()
            date_data = get_weather_data3(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    if not browser.current_window_handle: # Check if browser is still open
                        browser.quit()
                        browser = webdriver.Firefox()
                    date_data = get_weather_data3(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
                    logging.error("An error occurred while getting data for date %s: %s", current_date, e)
            if retries == max_retries:
                browser.quit()
                logging.error("Maximum number of retries reached for date %s", current_date)
                problematic_urls.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
def get_weather_data_for_dates5(start_year, start_month, start_day, end_year, end_month, end_day, browser):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    if not browser.execute_script("return window.chrome.webstore"): # Check if browser is still open
                        browser.quit()
                        print("---------quitting browser---------")
                        time.sleep(30)
                        browser = webdriver.Firefox()
                    date_data = get_weather_data3(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
                    logging.error("An error occurred while getting data for date %s: %s", current_date, e)
            if retries == max_retries:
                browser.quit()
                logging.error("Maximum number of retries reached for date %s", current_date)
                problematic_urls.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
def get_weather_data_for_dates4(start_year, start_month, start_day, end_year, end_month, end_day, browser):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day, browser)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    if not browser.execute_script("return window.chrome.webstore"): # Check if browser is still open
                        browser.quit()
                        time.sleep(30)
                        browser = webdriver.Chrome()
                    date_data = get_weather_data3(year, month, day, browser)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(30)
            if retries == max_retries:
                print("Maximum number of retries reached for date", current_date)
                problematic_urls.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
def get_weather_data_for_dates3(start_year, start_month, start_day, end_year, end_month, end_day):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            max_retries = 5
            retries = 0
            while retries < max_retries:
                try:
                    browser.quit()
                    time.sleep(60)
                    browser = webdriver.Chrome()
                    date_data = get_weather_data3(year, month, day)
                    weather_data[current_date.strftime("%Y-%m-%d")] = date_data
                    break
                except Exception as e:
                    retries += 1
                    time.sleep(60)
            if retries == max_retries:
                print("Maximum number of retries reached for date", current_date)
                problematic_urls.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
# Open browser
browser = webdriver.Chrome()
def get_weather_data_for_dates2(start_year, start_month, start_day, end_year, end_month, end_day):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
  
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            print("An error occurred while getting data for date", current_date, ":", e)
            problematic_urls.append(current_date.strftime("%Y-%m-%d"))
            browser.quit()
            time.sleep(60)
            browser = webdriver.Chrome()
        current_date += timedelta(days=1)
    browser.quit()
    return weather_data, problematic_urls

In [ ]:
# Open browser
browser = webdriver.Chrome()
    
def get_weather_data_for_dates(start_year, start_month, start_day, end_year, end_month, end_day):
    
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    end_date = datetime(end_year, end_month, end_day)
    while current_date <= end_date:
        try:
            year, month, day = current_date.year, current_date.month, current_date.day
            date_data = get_weather_data3(year, month, day)
            weather_data[current_date.strftime("%Y-%m-%d")] = date_data
        except Exception as e:
            print("An error occurred while getting data for date", current_date, ":", e)
        current_date += timedelta(days=1)
    #Close browser
    browser.quit()
    return weather_data